In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import yfinance as yf
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio
pio.renderers.default ='iframe'

In [2]:
import warnings
warnings.simplefilter('ignore',category=FutureWarning)

In [3]:
pd.set_option("display.max_columns", None)  # Show all columns
pd.set_option("display.width", None)        # No limit on line width
pd.set_option("display.max_colwidth", None) # Show full content in each column

In [4]:
def make_graph(stock_data, revenue_data, stock):
    fig = make_subplots(rows=2, cols=1, shared_xaxes=True, subplot_titles=("Historical Share Price", "Historical Revenue"), vertical_spacing = .3)
    stock_data_specific = stock_data[stock_data.Date <= '2021-06-14']
    revenue_data_specific = revenue_data[revenue_data.Date <= '2021-04-30']
    fig.add_trace(go.Scatter(x=pd.to_datetime(stock_data_specific.Date), y=stock_data_specific.Close.astype("float"), name="Share Price"), row=1, col=1)

    revenue_data_specific["Revenue"] = revenue_data_specific["Revenue"].replace(r'[\$,]', '', regex=True).astype(
        "float")
    fig.add_trace(
        go.Scatter(x=pd.to_datetime(revenue_data_specific.Date), y=revenue_data_specific.Revenue, name="Revenue"),
        row=2, col=1)
    fig.update_xaxes(title_text="Date", row=1, col=1)
    fig.update_xaxes(title_text="Date", row=2, col=1)
    fig.update_yaxes(title_text="Price ($US)", row=1, col=1)
    fig.update_yaxes(title_text="Revenue ($US Millions)", row=2, col=1)
    fig.update_layout(showlegend=False,
    height=900,
    title=stock,
    xaxis_rangeslider_visible=True)
    fig.show()
    from IPython.display import display, HTML
    fig_html = fig.to_html()
    display(HTML(fig_html))

In [5]:
tesla = yf.Ticker('TSLA')
tesla_data = tesla.history(period='max')
tesla_data = tesla_data.reset_index()
print(tesla_data.head())

                       Date      Open      High       Low     Close  \
0 2010-06-29 00:00:00-04:00  1.266667  1.666667  1.169333  1.592667   
1 2010-06-30 00:00:00-04:00  1.719333  2.028000  1.553333  1.588667   
2 2010-07-01 00:00:00-04:00  1.666667  1.728000  1.351333  1.464000   
3 2010-07-02 00:00:00-04:00  1.533333  1.540000  1.247333  1.280000   
4 2010-07-06 00:00:00-04:00  1.333333  1.333333  1.055333  1.074000   

      Volume  Dividends  Stock Splits  
0  281494500        0.0           0.0  
1  257806500        0.0           0.0  
2  123282000        0.0           0.0  
3   77097000        0.0           0.0  
4  103003500        0.0           0.0  


In [6]:
url = " https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0220EN-SkillsNetwork/labs/project/revenue.htm"

html_data = requests.get(url).text

In [7]:
soup = BeautifulSoup(html_data,"html.parser")

In [8]:
tesla_revenue = pd.DataFrame(columns=["Date","Revenue"])

for row in soup.find('tbody').find_all("tr"):
    col = row.find_all("td")
    date = col[0].text
    revenue = col[1].text

    tesla_revenue = pd.concat([tesla_revenue, pd.DataFrame(
        {"Date": [date],
        "Revenue": [revenue]})], ignore_index=True)


In [9]:
tesla_revenue["Revenue"] = tesla_revenue['Revenue'].str.replace(r'[\$,]', '', regex=True).astype("float")


tesla_revenue.dropna(inplace=True)

tesla_revenue = tesla_revenue[tesla_revenue['Revenue'] != ""]

In [10]:
print(tesla_revenue.tail())

    Date  Revenue
8   2013   2013.0
9   2012    413.0
10  2011    204.0
11  2010    117.0
12  2009    112.0


In [11]:
# Extracting GameStop(GME) stock data
gme = yf.Ticker('GME')
gme_data = gme.history(period='max')
gme_data = gme_data.reset_index()
print(gme_data.head())

                       Date      Open      High       Low     Close    Volume  \
0 2002-02-13 00:00:00-05:00  1.620128  1.693350  1.603296  1.691666  76216000   
1 2002-02-14 00:00:00-05:00  1.712708  1.716074  1.670627  1.683251  11021600   
2 2002-02-15 00:00:00-05:00  1.683250  1.687458  1.658001  1.674834   8389600   
3 2002-02-19 00:00:00-05:00  1.666418  1.666418  1.578047  1.607504   7410400   
4 2002-02-20 00:00:00-05:00  1.615920  1.662210  1.603296  1.662210   6892800   

   Dividends  Stock Splits  
0        0.0           0.0  
1        0.0           0.0  
2        0.0           0.0  
3        0.0           0.0  
4        0.0           0.0  


In [13]:
#Using request library to download the webpage
url2 = " https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0220EN-SkillsNetwork/labs/project/stock.html"
html_data2 = requests.get(url2).text

In [14]:
#Using BeautifulSoup to parse the html data
soup = BeautifulSoup(html_data2,"html5lib")

In [15]:
gme_revenue = pd.DataFrame(columns =["Date","Revenue"])

for row in soup.find("tbody").find_all("tr"):
    col = row.find_all("td")
    date = col[0].text
    revenue = col[1].text


    gme_revenue = pd.concat([gme_revenue, pd.DataFrame({"Date" : [date], "Revenue": [revenue]})], ignore_index=True)

In [16]:

gme_revenue["Revenue"] = gme_revenue['Revenue'].str.replace(r'[\$,]', '', regex=True).astype("float")

In [17]:
gme_revenue.dropna(inplace=True)

In [18]:
print(gme_revenue.tail())


    Date  Revenue
11  2009   8806.0
12  2008   7094.0
13  2007   5319.0
14  2006   3092.0
15  2005   1843.0


In [ ]:
make_graph(tesla_data, tesla_revenue, 'Tesla')